<a href="https://colab.research.google.com/github/jin433027/first/blob/main/otoki_ver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **주식가치평가(오뚜기)**

**배당할인모형을 바탕으로 가치평가를 진행하기**

In [1]:
import pandas as pd

In [ ]:
fin = pd.read_csv('/content/inv_data_2024.csv')

## **1. 항상성장모형을 적용하면 주식가치는?**

$$
P_0 = \frac{D_0(1+g)}{k - g}
$$
*할인율(k)를 7%라고 가정

1) 현재의 주당배당금($D_0$)

In [ ]:
#오뚜기의 배당금(천원)
o_d_value = fin.loc[
    (fin['Symbol'] == 'A007310') &
    (fin['계정'] == '6000908016'),
    '2024'
].values[0]
o_d_value = float(o_d_value.replace(',', ''))
o_d = o_d_value*1000

s_c_value = fin.loc[
    (fin['Symbol'] == 'A007310') &
    (fin['계정'] == 'S430001300'),
    '2024'
].values[0]
s_c_value = float(s_c_value.replace(',', ''))

pre_d_value = o_d / s_c_value
pre_d_value

7723.367258591307

2) 배당성장률(g)

g = b $\times$ r = b $\times$ ROE


In [ ]:
#내부유보율 = (1-배당성향)
#배당성향 = DPS/EPS
eps_value = fin.loc[
    (fin['Symbol'] == 'A007310') &
    (fin['계정'] == '6000601002'),
    '2024'
].values[0]
eps_value = float(eps_value.replace(',', ''))
eps_value

retention = float(pre_d_value / eps_value)

b = 1 - retention
b

roe_value = fin.loc[
    (fin['Symbol'] == 'A007310') &
    (fin['계정'] == '6000312001'),
    '2024'
].values[0]
roe_value = float(roe_value)
roe_value = roe_value/100 #ROE가 %단위

g_value = b * roe_value
g_value

0.05012677968931874

3) 주식가치($P_0$)

In [ ]:
stock_price = pre_d_value *(1+g_value)/(0.07-g_value)
stock_price

408112.7598260082

## **2. 2기간성장모형을 적용하면 주식가치는?**
(4년 동안의 성장률이 5년 이후부터는 절반으로 줄어든다고 가정)

$$
P_0 = [\frac{D_0(1+g_1)}{k - g_1}][1-\frac{(1+g_1)^N}{(1+k)^N}]+\frac{D_0(1+g_1)^N(1+g_2)}{(k - g_2)(1+k)^N}
$$

In [ ]:
#5년이후부터 성장률이 절반으로 줄어든다는 가정
g2_value = float(g_value / 2)
g2_value

n = float(4)
k = float(0.07)

In [ ]:
#고성장시기
high_growth = (pre_d_value * (1 + g_value) / (k - g_value)) \
              * (1 - ((1 + g_value) ** n) / ((1 + k) ** n))

#5년이후 저성장시기
Dn_plus_1 = pre_d_value * (1 + g_value) ** n * (1 + g2_value)
low_growth = Dn_plus_1 / (k - g2_value) / ((1 + k) ** n)

#최종
period_pre_value = high_growth + low_growth
period_pre_value

192936.90198287778

## **3. 유사기업인 농심, 삼양과 PER를 비교하여 가치를 평가한다면?**

$$
P_T = PER_C \times EPS_T
$$

In [ ]:
#농심의 PER
nongsim_per = fin.loc[
    (fin['Symbol'] == 'A004370') &
    (fin['계정'] == '6000701001'),
    '2024'
].values[0]
nongsim_per = float(nongsim_per.replace(',', ''))
nongsim_per

#삼양식품의 PER
samyang_per = fin.loc[
    (fin['Symbol'] == 'A003230') &
    (fin['계정'] == '6000701001'),
    '2024'
].values[0]
samyang_per = float(samyang_per.replace(',', ''))
samyang_per

#경쟁기업의 PER
peer_per = (nongsim_per + samyang_per) / 2
peer_per

#오뚜기의 EPS
otoki_eps = fin.loc[
    (fin['Symbol'] == 'A007310') &
    (fin['계정'] == '6000601002'),
    '2024'
].values[0]
otoki_eps = float(otoki_eps.replace(',', ''))
otoki_eps

#오뚜기의 PER 주식가치평가
o_per_stock_value = otoki_eps * peer_per
o_per_stock_value

607975.1000000001

In [ ]:
#2024년 오뚜기 주식의 종가
o_closing = fin.loc[
    (fin['Symbol'] == 'A007310') &
    (fin['계정'] == 'S430000200'),
    '2024'
].values[0]
o_closing = float(o_closing.replace(',', ''))
o_closing

395500.0

In [ ]:
#최종 : PER 주가가치와 종가와 비교
if o_per_stock_value > o_closing :
  print("PER 비교시 주가는 종가에 비해 저평가되었다")
else:
  print("PER 비교시 주가는 종가에 비해 고평가되었다")

PER 비교시 주가는 종가에 비해 저평가되었다
